# 简介

[wrapt](https://wrapt.readthedocs.io/en/master/)是一个简化装饰器编写的库,它提供了`FunctionWrapper`和`ObjectProxy`两个类,可以用来编写更加复杂的装饰器

# 安装

In [1]:
! pip install wrapt

  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl (37 kB)


# 函数装饰器

## 基本用法

下面这个例子定义了一个装饰器`my_decorator`,它接受一个函数作为参数,并返回一个新的函数,这个新的函数在调用原函数之前会输出`start`,在调用原函数之后会输出`end`

In [2]:
import wrapt
@wrapt.decorator
def my_decorator(wrapped, instance, args, kwargs):
    print("Start")
    result = wrapped(*args, **kwargs)
    print("End")
    return result

@my_decorator
def add(a, b):
    print("Add")
    return a + b

print(add(1, 2))

Start
Add
End
3


## 多个装饰器

下面这个例子用`wrapt`重现了上面的`decorator1`和`decorator2`的功能

In [ ]:
import wrapt

@wrapt.decorator
def decorator1(wrapped, instance, args, kwargs):
    print("Decorator 1")
    return wrapped(*args, **kwargs)

@wrapt.decorator
def decorator2(wrapped, instance, args, kwargs):
    print("Decorator 2")
    return wrapped(*args, **kwargs)

@decorator1
@decorator2
def my_function():
    print("Inside the function")

my_function()

## 装饰器参数

下面这个例子用`wrapt`实现了上面的`repeat`装饰器

In [ ]:
import wrapt

def repeat(n):
    @wrapt.decorator
    def decorator(wrapped, instance, args, kwargs):
        print(f"Repeat {n} times")
        for _ in range(n):
            result = wrapped(*args, **kwargs)
        return result
    return decorator

@repeat(3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")

# 类装饰器

## 基本用法

下面这个例子定义了一个类装饰器`@my_decorator`,它会在被包装的类初始化(`__init__`方法被调用)前打印`装饰器被调用`这句话

In [5]:
import wrapt


@wrapt.decorator
def my_decorator(wrapped, instance, args, kwargs):
    print("装饰器被调用")
    return wrapped(*args, **kwargs)

@my_decorator
class MyClass:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        print(f"Hello, {self.name}!")

obj = MyClass("John")
# obj.say_hello()

装饰器被调用


## 新增类的实例方法

下面这个例子定义了一个类装饰器`@add_method`,它会在被包装的类中新增一个方法`new_method`

In [30]:
import wrapt

@wrapt.decorator
def add_method(wrapped, instance, args, kwargs):
    class NewClass(wrapped):

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)

        def new_method(self):
            print("这是由装饰器添加的方法")
    return NewClass(*args, **kwargs)

@add_method
class MyClass:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        print("这是原始类自带的方法")

obj = MyClass("John")
obj.new_method()
obj.say_hello()

这是由装饰器添加的方法
这是原始类自带的方法


## 新增类的静态方法


下面这个例子定义了一个类装饰器`@add_static_method`,它会在被包装的类中新增一个静态方法`new_static_method`

In [33]:
import wrapt

@wrapt.decorator
def add_static_method(wrapped, instance, args, kwargs):
    def new_static_method():
        print("这是由装饰器添加的一个静态方法")
    wrapped.new_static_method = staticmethod(new_static_method)
    return wrapped(*args, **kwargs)

@add_static_method
class MyClass:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        print("这是原始类自带的方法")

#obj = MyClass("John")
# obj.say_hello()
# 这个有问题,装饰器本质上是一个函数,如果这个装饰器函数没被调用,那装饰逻辑就不会生效,我用装饰器给类添加静态方法却要求类实例化后才能调用,这没啥意义
MyClass.new_static_method()


AttributeError: type object 'MyClass' has no attribute 'new_static_method'